# Automated Essay Scoring

- applying NLP and Regressor model to analyse the model
- Dataset is from kaggle 

## importing the library

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\SAMEER\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\SAMEER\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## importing the dataset

In [2]:
train_data = pd.read_table('Data/training_set_rel3.tsv',encoding = "ISO-8859-1")

In [3]:
train_data.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train_data_except_set2 = train_data.loc[train_data.essay_set != 2,['essay_set','essay','domain1_score']]

In [5]:
train_data_except_set2.columns = ['essay_set','essay','score']

In [6]:
train_data_set2_1 = train_data.loc[train_data['essay_set'] == 2 ,['essay_set','essay','domain1_score']]

In [7]:
train_data_set2_1['essay_set'] = "2_1"

In [8]:
train_data_set2_1.columns = ['essay_set','essay','score']

In [9]:
train_data_set2_2 = train_data.loc[train_data['essay_set'] == 2 ,['essay_set','essay','domain2_score']]

In [10]:
train_data_set2_2['essay_set'] = "2_2"

In [11]:
train_data_set2_2.columns = ['essay_set','essay','score']

In [12]:
train_data_except_set2.head()

,essay_set,essay,score
0,1,"Dear local newspaper, I think effects computer...",8
1,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,1,"Dear @LOCATION1, I know having computers has a...",8


In [13]:
train_data_set2_2.head()

,essay_set,essay,score
1783,2_2,Certain materials being removed from libraries...,4.0
1784,2_2,Write a persuasive essay to a newspaper reflec...,1.0
1785,2_2,Do you think that libraries should remove cert...,3.0
1786,2_2,"In @DATE1's world, there are many things found...",4.0
1787,2_2,In life you have the 'offensive things'. The l...,4.0


In [14]:
train_data_set2_2.head()

,essay_set,essay,score
1783,2_2,Certain materials being removed from libraries...,4.0
1784,2_2,Write a persuasive essay to a newspaper reflec...,1.0
1785,2_2,Do you think that libraries should remove cert...,3.0
1786,2_2,"In @DATE1's world, there are many things found...",4.0
1787,2_2,In life you have the 'offensive things'. The l...,4.0


In [15]:
train_data_format = pd.concat([train_data_except_set2,train_data_set2_2,train_data_set2_2])

In [16]:
train_data_format.shape

(14776, 3)

In [17]:
def get_distinct_word_count(text):
    word_list = word_tokenize(text)
    return len(set(word_list))

In [18]:
#tokenize the word
def get_word_count_token(text):
    word_list = word_tokenize(text)
    return len(word_list)

In [19]:
#tokenize the sentence
def get_sentence_count(text):
    sent_list = sent_tokenize(text)
    return len(sent_list)

In [20]:
import re
def get_character_count(text):
    return len(text)

#removing the space
def get_word_count(text):
    return len(re.findall(r"\s", text))+1

In [21]:
train_data_format['length'] = train_data_format['essay'].apply(get_character_count)
train_data_format['word_count'] = train_data_format['essay'].apply(get_word_count)

In [22]:
train_data_format['distinct_word'] = train_data_format['essay'].apply(get_distinct_word_count)

In [23]:
train_data_format['sentence_count'] = train_data_format['essay'].apply(get_sentence_count)

### Inserting the above line in dataset

In [24]:
train_data_format.head(5)

,essay_set,essay,score,length,word_count,distinct_word,sentence_count
0,1,"Dear local newspaper, I think effects computer...",8.0,1875,338,181,16
1,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9.0,2288,419,209,20
2,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7.0,1541,279,161,14
3,1,"Dear Local Newspaper, @CAPS1 I have found that...",10.0,3165,524,267,27
4,1,"Dear @LOCATION1, I know having computers has a...",8.0,2569,465,211,30


## Importing the test set

In [25]:
test_data = pd.read_table('Data/valid_set.tsv',encoding = "ISO-8859-1")

In [26]:
test_data.head()

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,NaN
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,NaN
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,NaN
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,NaN
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,NaN


### Cleaning the dataset

In [27]:
test_data_except_set2 = test_data.loc[test_data.essay_set != 2,['essay_set','essay','domain1_predictionid']]
test_data_except_set2.columns = ['essay_set','essay','predictionid']
test_data_set2_1 = test_data.loc[test_data['essay_set'] == 2 ,['essay_set','essay','domain1_predictionid']]
test_data_set2_1['essay_set'] = "2_1"
test_data_set2_1.columns = ['essay_set','essay','predictionid']
test_data_set2_2 = test_data.loc[test_data['essay_set'] == 2 ,['essay_set','essay','domain2_predictionid']]
test_data_set2_2['essay_set'] = "2_2"
test_data_set2_2.columns = ['essay_set','essay','predictionid']

In [28]:
test_data_set2_2.columns

Index(['essay_set', 'essay', 'predictionid'], dtype='object')

In [29]:
test_data_except_set2.head()

,essay_set,essay,predictionid
0,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788
1,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789
2,1,"Dear Local newspaper, Have you been spending a...",1790
3,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791
4,1,"Dear newspaper, I strongly believe that comput...",1792


In [30]:
test_data_set2_1.head()

,essay_set,essay,predictionid
589,2_1,"Do u believe there are books, music, magizines...",6578
590,2_1,I strongly believe that there are some materia...,6580
591,2_1,"Do you think that certain books, movies, magaz...",6582
592,2_1,Censorship in libraries should definetly be al...,6584
593,2_1,Many books are helpful as you @MONTH1 know by ...,6586


In [31]:
test_data_set2_2.head()

,essay_set,essay,predictionid
589,2_2,"Do u believe there are books, music, magizines...",6579.0
590,2_2,I strongly believe that there are some materia...,6581.0
591,2_2,"Do you think that certain books, movies, magaz...",6583.0
592,2_2,Censorship in libraries should definetly be al...,6585.0
593,2_2,Many books are helpful as you @MONTH1 know by ...,6587.0


In [32]:
test_data_format = pd.concat([test_data_except_set2,test_data_set2_1,test_data_set2_2])

In [33]:
train_data_format.head()

,essay_set,essay,score,length,word_count,distinct_word,sentence_count
0,1,"Dear local newspaper, I think effects computer...",8.0,1875,338,181,16
1,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9.0,2288,419,209,20
2,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7.0,1541,279,161,14
3,1,"Dear Local Newspaper, @CAPS1 I have found that...",10.0,3165,524,267,27
4,1,"Dear @LOCATION1, I know having computers has a...",8.0,2569,465,211,30


In [34]:
test_data_format.head()

,essay_set,essay,predictionid
0,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788.0
1,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789.0
2,1,"Dear Local newspaper, Have you been spending a...",1790.0
3,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791.0
4,1,"Dear newspaper, I strongly believe that comput...",1792.0


In [35]:
X = train_data_format['essay']


In [36]:
y = train_data_format['score']


In [37]:
sort_by_id = test_data_format.sort_values(by=['predictionid'], axis=0,inplace=True)

In [38]:
X_test = test_data_format['essay']

In [39]:
test_data_format.shape

(4818, 3)

In [40]:
X.head()

0    Dear local newspaper, I think effects computer...
1    Dear @CAPS1 @CAPS2, I believe that using compu...
2    Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...
3    Dear Local Newspaper, @CAPS1 I have found that...
4    Dear @LOCATION1, I know having computers has a...
Name: essay, dtype: object

In [41]:
y.head()

0     8.0
1     9.0
2     7.0
3    10.0
4     8.0
Name: score, dtype: float64

### Convert a collection of text documents to a matrix of token counts

In [42]:
vect = CountVectorizer()

In [43]:
vect.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [44]:
vect_dtm = vect.transform(X)

In [45]:
vect_dtm

<14776x38218 sparse matrix of type '<class 'numpy.int64'>'
	with 1680808 stored elements in Compressed Sparse Row format>

### Applying the random forest

In [46]:
randomRegressor = RandomForestRegressor(n_estimators=100)

In [54]:
rndfrst = randomRegressor.fit(vect_dtm,y)

In [48]:
test_vect_dtm = vect.transform(X_test)

In [49]:
test_vect_dtm

<4818x38218 sparse matrix of type '<class 'numpy.int64'>'
	with 544400 stored elements in Compressed Sparse Row format>

### Predicting for test case

In [50]:
y_pred_class = randomRegressor.predict(test_vect_dtm)

In [51]:
y_pred_class_round = [int(i) for i in y_pred_class]
predictionid_list =  [ int(id) for id in test_data_format['predictionid']]

### Saving the predicted data to the new csv file

In [53]:
pd.DataFrame({'prediction_id':predictionid_list,'predicted_score':y_pred_class_round}).to_csv('Output/submission_2.csv',columns=['prediction_id','predicted_score'],index=False)